In [1]:
import pandas as pd
import numpy as np
import nose.tools

# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

### Problem 1. Read the dataset (2 points)
The dataset [here](http://archive.ics.uci.edu/ml/datasets/Auto+MPG) contains information about fuel consumption in different cars.

Click the "Data Folder" link and read `auto_mpg.data` into Python. You can download it, if you wish, or you can read it directly from the link.

Give meaningful (and "Pythonic") column names, as per the `auto_mpg.names` file:
1. mpg
2. cylinders
3. displacement
4. horsepower
5. weight
6. acceleration
7. model_year
8. origin
9. car_name

In [2]:
mpg_data = pd.read_fwf("./data/auto-mpg.data", header = None)
mpg_data.columns = ["mpg", "cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year", "origin", "car_name"]

In [3]:
mpg_data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,"""amc rebel sst"""
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,"""ford torino"""
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,"""ford mustang gl"""
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,"""vw pickup"""
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,"""dodge rampage"""
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,"""ford ranger"""


In [4]:
nose.tools.assert_is_not_none(mpg_data)

Print the first 4 rows in the dataset to get a feel of what it looks like:

In [5]:
mpg_data.head(4)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,"""amc rebel sst"""


### Problem 2. Inspect the dataset (1 point)
Write a function which accepts a dataset and returns the number of observations and features in it, like so: 

``` 10 observations on 15 features```

Where 10 and 15 should be replaced with the real numbers. Test your function with the `auto_mpg` dataset.

Make sure the function works with other datasets (don't worry about "1 features" or "1 observations", just leave it as it is).

In [6]:
def observations_and_features(dataset):
    """
    Returns the number of observations and features in the provided dataset
    """
    observations = dataset.shape[0]
    features = dataset.shape[1]
#     return f'{observations} observations on {features} features'
    return "{} observations on {} features".format(observations, features)

In [7]:
print(observations_and_features(mpg_data))

398 observations on 9 features


Inspect the data types for each column.

In [8]:
mpg_data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model_year        int64
origin            int64
car_name         object
dtype: object

### Problem 3. Correct errors (1 point)
The `horsepower` column looks strange. It's a string but it must be a floating-point number. Find out why this is so and convert it to floating-point number.

In [9]:
# It is written in the description that there are 6 missing values in here and this appears to be true
# I've manualle checked that it is "?" sign, but seems more accurate to use a more "dynamic" check

mpg_data["horsepower"][mpg_data["horsepower"].str.len() < 3]

32     ?
126    ?
330    ?
336    ?
354    ?
374    ?
Name: horsepower, dtype: object

In [10]:
mpg_data["horsepower"] = mpg_data["horsepower"].replace("?", np.nan)

In [11]:
mpg_data["horsepower"][mpg_data["horsepower"].isna()]

32     NaN
126    NaN
330    NaN
336    NaN
354    NaN
374    NaN
Name: horsepower, dtype: object

In [12]:
mpg_data["horsepower"] = mpg_data["horsepower"].astype("float64")

In [13]:
mpg_data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight          float64
acceleration    float64
model_year        int64
origin            int64
car_name         object
dtype: object

In [14]:
nose.tools.assert_equal(mpg_data.horsepower.dtype, "float64")

### Problem 4. Missing values: inspection (1 point)
We saw that the `horsepower` column contained null values. Display the rows which contain those values. Assign the resulting dataframe to the `unknown_hp` variable.

In [15]:
def get_unknown_hp(dataframe):
    """
    Returns the rows in the provided dataframe where the "horsepower" column is NaN
    """
    unknown_hp = dataframe["horsepower"][dataframe["horsepower"].isna()]
    return unknown_hp

In [16]:
cars_with_unknown_hp = get_unknown_hp(mpg_data)
print(cars_with_unknown_hp)

32    NaN
126   NaN
330   NaN
336   NaN
354   NaN
374   NaN
Name: horsepower, dtype: float64


### Problem 5. Missing data: correction (1 point)
It seems like the `NaN` values are a small fraction of all values. We can try one of several things:
* Remove them
* Replace them (e.g. with the mean power of all cars)
* Look up the models on the internet and try our best guess on the power

The third one is probably the best but the first one will suffice since these records are too few. Remove those values. Save the dataset in the same `mpg_data` variable. Ensure there are no more `NaN`s.

In [17]:
mpg_data = mpg_data.dropna()

In [18]:
mpg_data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,"""amc rebel sst"""
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,"""ford torino"""
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1,"""ford mustang gl"""
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,"""vw pickup"""
395,32.0,4,135.0,84.0,2295.0,11.6,82,1,"""dodge rampage"""
396,28.0,4,120.0,79.0,2625.0,18.6,82,1,"""ford ranger"""


In [19]:
mpg_data[mpg_data["horsepower"].isna()]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name


In [20]:
nose.tools.assert_equal(len(get_unknown_hp(mpg_data)), 0)

### Problem 6. Years of production (1 + 1 points)
Display all unique model years. Assign them to the variable `model_years`.

In [21]:
mpg_data["model_year"]

0      70
1      70
2      70
3      70
4      70
       ..
393    82
394    82
395    82
396    82
397    82
Name: model_year, Length: 392, dtype: int64

In [22]:
def get_unique_model_years(dataframe):
    """
    Returns the unique values of the "model_year" column
    of the dataframe
    """
    model_years = dataframe["model_year"].unique()
    return model_years

In [23]:
model_years = get_unique_model_years(mpg_data)
print(model_years)

[70 71 72 73 74 75 76 77 78 79 80 81 82]


In [24]:
type(model_years)

numpy.ndarray

These don't look so good. Convert them to real years, like `70 -> 1970, 71 -> 1971`. Replace the column values in the dataframe.

In [25]:
mpg_data.loc[:, "model_year"] = mpg_data["model_year"] + 1900

C:\Users\ptsvetkov\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [26]:
mpg_data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight          float64
acceleration    float64
model_year        int64
origin            int64
car_name         object
dtype: object

In [27]:
mpg_data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,1970,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.0,11.5,1970,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.0,11.0,1970,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.0,12.0,1970,1,"""amc rebel sst"""
4,17.0,8,302.0,140.0,3449.0,10.5,1970,1,"""ford torino"""
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,1982,1,"""ford mustang gl"""
394,44.0,4,97.0,52.0,2130.0,24.6,1982,2,"""vw pickup"""
395,32.0,4,135.0,84.0,2295.0,11.6,1982,1,"""dodge rampage"""
396,28.0,4,120.0,79.0,2625.0,18.6,1982,1,"""ford ranger"""


In [28]:
model_years = get_unique_model_years(mpg_data)
print(model_years)

[1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982]


### Problem 7. Exploration: low-power cars (1 point)
The data looks quite good now. Let's try some exploration.

Write a function to find the cars which have the smallest number of cylinders and print their model names. Return a list of car names.

In [29]:
def get_model_names_smallest_cylinders(dataframe):
    """
    Returns the names of the cars with the smallest number of cylinders
    """
    car_names = dataframe[dataframe["cylinders"] == dataframe["cylinders"].min()]["car_name"].to_numpy()
    return car_names

In [30]:
car_names = get_model_names_smallest_cylinders(mpg_data)
print(car_names)
nose.tools.assert_true(car_names.shape == (4,) or car_names.shape == (4, 1))

['"mazda rx2 coupe"' '"maxda rx3"' '"mazda rx-4"' '"mazda rx-7 gs"']


### Problem 8. Exploration: correlations (1 point)
Finally, let's see some connections between variables. These are also called **correlations**.

Find how to calculate correlations between different columns using `pandas`.

**Hint:** The correlation function in `pandas` returns a `DataFrame` by default. You need only one value from it.

Create a function which accepts a dataframe and two columns and prints the correlation coefficient between those two columns.

In [31]:
def calculate_correlation(dataframe, first_column, second_column):
    """
    Calculates and returns the correlation coefficient between the two columns in the dataframe.
    """
    correlation = dataframe[first_column].corr(dataframe[second_column])
    return correlation

In [32]:
hp_weight = calculate_correlation(mpg_data, "horsepower", "weight")
print("Horsepower:Weight correlation coefficient:", hp_weight)
nose.tools.assert_almost_equal(hp_weight, 0.864537737574, delta = 0.01)


Horsepower:Weight correlation coefficient: 0.8645377375741441
